In [1]:
import cv2
import os
import numpy as np

import tensorflow as tf
import keras
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, InputLayer, Flatten, Dense
from sklearn.model_selection import train_test_split

from model import MBConv, SEBlock


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_path = 'train'
labels = os.listdir(train_path)
X = []
y = []
for label in labels:
    path = os.path.join(train_path, label)
    image_names = os.listdir(path)

    for image_name in image_names:
        img = cv2.imread(os.path.join(path, image_name))
        #gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #processed = cv2.normalize(gray_img, None, 0, 255, cv2.NORM_MINMAX)
        #img = np.concatenate([img, processed[:, :, np.newaxis]], axis=2)
        X.append(img / 255.)
        y.append(label)
y = list(map(lambda x: int(x) - 1, y))

In [4]:
y = tf.one_hot(y, 3)
X = tf.convert_to_tensor(X, dtype=tf.float32)

In [5]:
batch_size = 16

In [6]:
X = tf.random.shuffle(X, seed=43)
y = tf.random.shuffle(y, seed=43)

In [7]:
pivot = int(len(X) * 0.9)
pivot

405

In [8]:
X_train = X[:pivot]
X_valid = X[pivot:]
y_train = y[:pivot]
y_valid = y[pivot:]
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(405, 250, 250, 3) (405, 3)
(46, 250, 250, 3) (46, 3)


In [9]:
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(250, 250, 3), padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu", padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu", padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 250, 250, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 125, 125, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 62, 62, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 128)      0

In [10]:
from tensorflow.keras.metrics import Accuracy, CategoricalCrossentropy

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])

In [11]:
history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_valid, y_valid))

Epoch 1/10


: 

: 